In [1]:
import pandas as pd
import numpy as np
import os
os.listdir('../input')

['df_ref.csv',
 'train.csv',
 'submission_popular.csv',
 'item_metadata.csv',
 'df_full_except_noinfo.csv',
 'test.csv',
 'dwell_df.csv',
 'baseline_sorting.csv',
 'dwell_time_agg.csv',
 'test2.csv']

In [2]:
path = '../input/'
train = pd.read_csv(path + 'train.csv')
test2 = pd.read_csv(path + 'test2.csv')

In [60]:
print('train shape : ', train.shape)
print('test2 shape : ', test2.shape)

train shape :  (15932992, 12)
test2 shape :  (3528762, 13)


In [3]:
test_answer = test2['answer']
test2.drop('answer', inplace = True, axis = 1)

In [4]:
df = pd.concat([train, test2])
df.shape

(19461754, 12)

# except no info

In [5]:
no_info = df.groupby(['user_id', 'session_id'])['step'].max()
no_info = pd.DataFrame(no_info[no_info == 1]).reset_index()
no_info.head(3)

,user_id,session_id,step
0,0006W0R5A5V8,85501a9f27263,1
1,00071784XQ6B,9617600e1ba7c,1
2,0008B0X0HC39,1b703eda68d85,1


In [6]:
df = df[~df.session_id.isin(no_info.session_id)].reset_index(drop = True)
df.shape

(19214316, 12)

# reference count

In [7]:
#df.to_csv(path + 'df_full_except_noinfo.csv', index = False)

In [27]:
def only_number(string):
    """reference에서 숫자인 것만 뽑아 int type으로 변환"""
    try:
        int(string)
        return string
    except ValueError:
        return np.nan

In [28]:
df.reference = df.reference.apply(only_number)

In [51]:
df_ref2 = df.groupby('user_id')['reference'].value_counts().to_frame().rename(columns = {'reference' : 'ref_count'}).reset_index()
df_ref2.head(3)

,user_id,reference,ref_count
0,0001VQMGUI65,3133074,11
1,0001VQMGUI65,2019467,1
2,0001VQMGUI65,4521140,1


In [53]:
#df_ref2.to_csv(path + 'df_ref2.csv', index = False)
#del df_ref2

In [35]:
aa = df.groupby(['user_id','session_id'])['reference'].value_counts().to_frame().rename(columns = {'reference' : 'ref_count'})
aa.reset_index(inplace = True)
aa.head(3)

,user_id,session_id,reference,ref_count
0,0001VQMGUI65,466687f1875ee,3133074,11
1,0001VQMGUI65,466687f1875ee,2019467,1
2,0001VQMGUI65,466687f1875ee,4521140,1


In [43]:
aa['rank'] = aa.groupby(['user_id','session_id'])['ref_count'].rank(ascending = False)
aa.head(3)

,user_id,session_id,reference,ref_count,rank
0,0001VQMGUI65,466687f1875ee,3133074,11,1.0
1,0001VQMGUI65,466687f1875ee,2019467,1,3.5
2,0001VQMGUI65,466687f1875ee,4521140,1,3.5


# make submission file

In [55]:
test2['answer'] = test_answer

In [197]:
new_submission = test2[test2.answer == True][['user_id','session_id','timestamp','step','reference','impressions']]
#new_submission.to_csv(path + 'new_submission.csv', index = False)

In [60]:
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions
5,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,2059240|2033381|1724779|127131|399441|103357|1...
7,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480|4476010|3843244|3833012|9017890|198100...
11,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,3565720|2947584|4115018|2039671|3836538|801409...


# sort by ref count

In [200]:
new_submission = pd.read_csv(path + 'new_submission.csv')
new_submission['impressions'] = new_submission['impressions'].apply(lambda x : x.split('|'))
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ..."


In [91]:
ref_count_sort = aa.groupby(['user_id', 'session_id'])['reference'].apply(lambda x : ' '.join(x)).to_frame().reset_index()
ref_count_sort = ref_count_sort[(ref_count_sort.user_id.isin(new_submission.user_id)) & (ref_count_sort.session_id.isin(new_submission.session_id))]
ref_count_sort.head()

,user_id,session_id,reference
6,0008BO33KUQ0,2d0e2102ee0dc,452021 507861
12,000GO9NY6P4M,c712ede188d00,1618677
14,000IRHJS2DL9,f6ffffd20d43d,2547840 346166 2285010 4552802
18,000OWRCYEHKT,53e84da5c2dad,7795438
19,000VBY1D6BP8,033fddaaa99af,32246 5592656


In [115]:
ref_count_sort.reference = ref_count_sort.reference.apply(lambda x : x.split(' '))
ref_count_sort.rename(columns = {'reference' : 'ref_count_sorted'}, inplace = True)
ref_count_sort.head(3)

,user_id,session_id,ref_count_sorted
6,0008BO33KUQ0,2d0e2102ee0dc,"[452021, 507861]"
12,000GO9NY6P4M,c712ede188d00,[1618677]
14,000IRHJS2DL9,f6ffffd20d43d,"[2547840, 346166, 2285010, 4552802]"


In [202]:
new_submission = pd.merge(new_submission, ref_count_sort, on = ['user_id', 'session_id'], how = 'left')

In [203]:
def sort_ref(imp, ref) :
    my = []
    for rr in ref :
        if rr in imp :
            my.append(rr)
            imp.remove(rr)
    imp = my + imp
    return imp

In [204]:
#apply 
apply_index = new_submission[(new_submission.user_id.isin(ref_count_sort.user_id)) & (new_submission.session_id.isin(ref_count_sort.session_id))].index
new_submission.loc[apply_index, 'impressions'] = new_submission.loc[apply_index].apply(lambda x : sort_ref(x['impressions'], x['ref_count_sorted']), axis = 1)

In [170]:
def mrr(answer, imp) :
    myscore = 0
    
    for i,j in zip(answer, imp) :
        if i in j :
            myscore += 1/(j.index(i) + 1)
    print(myscore / len(imp))

In [205]:
new_submission.reference = new_submission.reference.apply(str)
mrr(new_submission['reference'], new_submission['impressions'])

0.7669310355806487


In [207]:
new_submission.impressions = new_submission.impressions.apply(lambda x : ' '.join(x))
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,1050068 2059240 3241426 2033381 1724779 127131...,"[1050068, 2059240, 3241426]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,[2817590]


In [225]:
ind = new_submission.loc[~new_submission.ref_count_sorted.isnull()].index
new_submission.loc[ind, 'ref_count_sorted'] = new_submission.loc[ind]['ref_count_sorted'].apply(lambda x : ' '.join(x))

In [229]:
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,1050068 2059240 3241426 2033381 1724779 127131...,1050068 2059240 3241426
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,2817590


In [230]:
#new_submission.to_csv(path + 'new_submission_baseline.csv', index = False)